# CellRanger output processing

This notebook goals is to recreate the cell x gene expression matrix from the [output](https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/output/overview) of [CellRanger 3.1](https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/what-is-cell-ranger)

In [1]:
import os
import pandas as pd
import numpy as np

### 1. Download the [Feature-Barcode Matrices](https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/output/matrices) from the dataset of your choice in the Market Exchange format

In this example, we will use data from 4 healthy PBMC donors: https://support.10xgenomics.com/single-cell-vdj/datasets/ (Application Note - A New Way of Exploring Immunity)

[Donor 1](https://support.10xgenomics.com/single-cell-vdj/datasets/3.0.2/vdj_v1_hs_aggregated_donor1)
[Donor 2](https://support.10xgenomics.com/single-cell-vdj/datasets/3.0.2/vdj_v1_hs_aggregated_donor2)
[Donor 3](https://support.10xgenomics.com/single-cell-vdj/datasets/3.0.2/vdj_v1_hs_aggregated_donor3)
[Donor 4](https://support.10xgenomics.com/single-cell-vdj/datasets/3.0.2/vdj_v1_hs_aggregated_donor4)

In [ ]:
#create directories to store the data
donors = ["donor"+str(i+1) for i in range(4)]
for d in donors:
    try:
        os.mkdir(d)
    except FileExistsError:
        pass

The following bash script does 3 things for each donor:
* Download the data
* Extract the tar file
* Unzip the 3 matrices


In [ ]:
%%bash
counter=1
while [ $counter -le 4 ]
do
cd 'donor'$counter
echo $PWD
curl -O 'http://cf.10xgenomics.com/samples/cell-vdj/3.0.2/vdj_v1_hs_aggregated_donor'$counter'/vdj_v1_hs_aggregated_donor'$counter'_filtered_feature_bc_matrix.tar.gz'
tar -xvf *.tar.gz
gunzip filtered_feature_bc_matrix/*.gz 
cd ..
((counter++))
done

### 3. Recreate and export the cell x gene expression matrix

The mapping_mtx indicates counts for each barcode/gene combination. We first create an empty expression matrix and then add count using the mapping matrix.

In [3]:
MATRIX_DIR = "filtered_feature_bc_matrix"

In [4]:
for donor in donors:
    if os.path.exists(os.path.join(donor, 'exp_mtx.h5')):
        continue
        
    print('Processing ', donor)
    #import features
    feat = pd.read_csv(os.path.join(donor, MATRIX_DIR, "features.tsv"), sep='\t', 
                   header=None, names= ['feature_ID', 'feature_name', 'feature_type'])
    print('number of features: ',feat.shape[0])
    
    #import barcodes
    bc = pd.read_csv(os.path.join(donor, MATRIX_DIR, "barcodes.tsv"), sep='\t', 
                 header=None, names= ['barcodes'])
    print('number of barcodes: ',bc.shape[0])
    
    #import exp matrix
    mtx = pd.read_csv(os.path.join(donor, MATRIX_DIR, "matrix.mtx"), sep='\t')
    #check that indices length in mtx match the number of features and barcodes
    len_feat, len_bc, len_mtx = [int(x) for x in mtx.iloc[1,0].split(' ')]
    if len_feat != feat.shape[0]:
        print('The number of features does not match the matrix file')
    if len_bc != bc.shape[0]:
        print('The number of barcodes does not match the matrix file')
    if len_mtx != mtx.shape[0]-2:
        print('The number of rows in the matrix file does not match its index')
    
    if os.path.exists(os.path.join(donor, 'mapping_mtx.h5')):
        print('Loading mapping coordinates...')
        mapping_mtx = pd.read_hdf(os.path.join(donor, 'mapping_mtx.h5'), key= 'data')
    else:    
        print('Creating mapping coordinates...')
        mapping_mtx = pd.DataFrame(mtx.iloc[2:,0].str.split(' ', expand=True).values, columns=['feat','bc','count'], dtype='int')
        mapping_mtx.to_hdf(os.path.join(donor, 'mapping_mtx.h5'), key= 'data')
    #create empty cell x gene matrix
    print('Creating cell x gene matrix...')
    exp_mtx = pd.DataFrame(data= 0, index= bc['barcodes'], columns= feat['feature_ID'])
    
    #add count data
    print('Mapping counts to cell x gene expression matrix...')
    for idx, coord in mapping_mtx.iterrows():
        if idx%1000000 == 0:
            print(idx, ' rows processed, ', len_mtx-idx, 'remaining...')
        exp_mtx.iloc[coord['bc']-1,coord['feat']-1] = coord['count']
        
    #exp_mtx.to_csv('exp_mtx.csv')
    print('Exporting expression matrix for ', donor, '...')
    exp_mtx.to_hdf(os.path.join(donor, 'exp_mtx.h5'), key= 'data')

Processing  donor3
number of features:  33602
number of barcodes:  54137
Loading mapping coordinates...
Creating cell x gene matrix...
Mapping counts to cell x gene expression matrix...
0  rows processed,  69912747 remaining...
1000000  rows processed,  68912747 remaining...
2000000  rows processed,  67912747 remaining...
3000000  rows processed,  66912747 remaining...
4000000  rows processed,  65912747 remaining...
5000000  rows processed,  64912747 remaining...
6000000  rows processed,  63912747 remaining...
7000000  rows processed,  62912747 remaining...
8000000  rows processed,  61912747 remaining...
9000000  rows processed,  60912747 remaining...
10000000  rows processed,  59912747 remaining...
11000000  rows processed,  58912747 remaining...
12000000  rows processed,  57912747 remaining...
13000000  rows processed,  56912747 remaining...
14000000  rows processed,  55912747 remaining...
15000000  rows processed,  54912747 remaining...
16000000  rows processed,  53912747 remaining.